# Transfomaciones de los datos
---

### Se importan librerías a usar

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Carga de los Datasets

In [60]:
# Lectura de los Csv
Acceso_internet_fijo_velocidadbajada_provincia=pd.read_csv('Datasets/Data_ingested/Accesos_a_Internet_fijo_por_velocidad_bajada_y_provincia.csv', encoding='utf-8')
Acceso_internet_fijo_velocidadbajada_localidad=pd.read_csv('Datasets/Data_ingested/Accesos_a_Internet_fijo_por_velocidad_de_bajada_y_localidad.csv', encoding='utf-8')
Conectividad_Servicio_Internet=pd.read_csv('Datasets/Data_ingested/Conectividad_al_servicio_de_Internet.csv', encoding='utf-8')
Internet_Accesos_por_tecnologia=pd.read_csv('Datasets/Data_ingested/Internet_Accesos-por-tecnologia.csv', encoding='utf-8')
Internet_Accesos_velocidad=pd.read_csv('Datasets/Data_ingested/Internet_Accesos-por-velocidad.csv', encoding='utf-8')
Internet_BAF=pd.read_csv('Datasets/Data_ingested/Internet_BAF.csv', encoding='utf-8')
Internet_ingresos = pd.read_csv('Datasets/Data_ingested/Internet_Penetracion.csv', encoding='utf-8')
Internet_Penetracion = pd.read_csv('Datasets/Data_ingested/Internet_Penetracion.csv', encoding='utf-8')
Listado_localidades_conectividad_internet=pd.read_csv('Datasets/Data_ingested/Listado_de_localidades_con_conectividad_a_internet.csv', encoding='utf-8')
velocidadInternet_anio = pd.read_csv('Datasets/Data_ingested/velocidad_internet_promedio.csv', encoding='utf-8')

#nombres de los dataframes para accederlos mas facilmente
names_dataframes = ['Acceso_internet_fijo_velocidad_bajada_provincia', 'Acceso_internet_fijo_velocidad_bajada_localidad', 'Conectividad_Servicio_Internet', 'Internet_Accesos_por_tecnologia',
'Internet_Accesos_velocidad', 'Internet_BAF', 'Internet_ingresos', 'Internet_Penetracion', 'Listado_localidades_conectividad_internet', 'velocidadInternet_anio']

####  Voy a incurrir en un analisis especifico para cuantificar la cantidad de nulos y establecere un porcentaje maximo de estos. De un 60% ya que al tener un plazo reducido de tiempo,se necesita rapidez

In [70]:
# Identificar los registros del DataFrame
registros = Acceso_internet_fijo_velocidadbajada_provincia.shape[0]

# Obtener la cantidad total de registros nulos en el DataFrame
registros_nulos = Acceso_internet_fijo_velocidadbajada_provincia.isnull().sum().sum()
# obtener el porcentaje de nulos en el dataframe
percent_of_nulls = round((registros_nulos / registros),0)

print(names_dataframes[0],'tiene un: ',percent_of_nulls,'% de nulos')

# Identificar los registros del DataFrame
registros1 = Acceso_internet_fijo_velocidadbajada_localidad.shape[0]
registros_nulos1 = Acceso_internet_fijo_velocidadbajada_localidad.isnull().sum().sum()
percent_of_nulls1 = round((registros_nulos / registros),0)
print(names_dataframes[1],'tiene un: ',percent_of_nulls1,'% de nulos')


# Identificar los registros del DataFrame
registros2 = Conectividad_Servicio_Internet.shape[0]
registros_nulos2 = Conectividad_Servicio_Internet.isnull().sum().sum()
percent_of_nulls2= round((registros_nulos / registros),0)
print(names_dataframes[2],'tiene un: ',percent_of_nulls2,'% de nulos')


# Identificar los registros del DataFrame
registros3 = Internet_Accesos_por_tecnologia.shape[0]
registros_nulos3 = Internet_Accesos_por_tecnologia.isnull().sum().sum()
percent_of_nulls3 = round((registros_nulos / registros),0)
print(names_dataframes[3],'tiene un: ',percent_of_nulls3,'% de nulos')


# Identificar los registros del DataFrame
registros4 = Internet_Accesos_velocidad.shape[0]
registros_nulos4 = Internet_Accesos_velocidad.isnull().sum().sum()
percent_of_nulls4 = round((registros_nulos / registros),0)
print(names_dataframes[4],'tiene un: ',percent_of_nulls4,'% de nulos')


# Identificar los registros del DataFrame
registros5 = Internet_BAF.shape[0]
registros_nulos5 = Internet_BAF.isnull().sum().sum()
percent_of_nulls5 = round((registros_nulos / registros),0)
print(names_dataframes[5],'tiene un: ',percent_of_nulls5,'% de nulos')



# Identificar los registros del DataFrame
registros6 = Internet_ingresos.shape[0]
registros_nulos6 = Internet_ingresos.isnull().sum().sum()
percent_of_nulls6 = round((registros_nulos / registros),0)
print(names_dataframes[6],'tiene un: ',percent_of_nulls6,'% de nulos')



# Identificar los registros del DataFrame
registros7 = Internet_Penetracion.shape[0]
registros_nulos7 = Internet_Penetracion.isnull().sum().sum()
percent_of_nulls7 = round((registros_nulos / registros),0)
print(names_dataframes[7],'tiene un: ',percent_of_nulls7,'% de nulos')



# Identificar los registros del DataFrame
registros8 = Listado_localidades_conectividad_internet.shape[0]
registros_nulos8 = Listado_localidades_conectividad_internet.isnull().sum().sum()
percent_of_nulls8 = round((registros_nulos / registros),0)
print(names_dataframes[8],'tiene un: ',percent_of_nulls8,'% de nulos')


# Identificar los registros del DataFrame
registros9 = velocidadInternet_anio.shape[0]
registros_nulos9 = velocidadInternet_anio.isnull().sum().sum()
percent_of_nulls9 = round((registros_nulos / registros),0)
print(names_dataframes[9],'tiene un: ',percent_of_nulls9,'% de nulos')


Acceso_internet_fijo_velocidad_bajada_provincia tiene un:  0.0 % de nulos
Acceso_internet_fijo_velocidad_bajada_localidad tiene un:  0.0 % de nulos
Conectividad_Servicio_Internet tiene un:  0.0 % de nulos
Internet_Accesos_por_tecnologia tiene un:  0.0 % de nulos
Internet_Accesos_velocidad tiene un:  0.0 % de nulos
Internet_BAF tiene un:  0.0 % de nulos
Internet_ingresos tiene un:  0.0 % de nulos
Internet_Penetracion tiene un:  0.0 % de nulos
Listado_localidades_conectividad_internet tiene un:  0.0 % de nulos
velocidadInternet_anio tiene un:  0.0 % de nulos


Acceso_internet_fijo_velocidad_bajada_localidad tiene un:  73.0 % de nulos


Conectividad_Servicio_Internet tiene un:  0.0 % de nulos


Internet_Accesos_por_tecnologia tiene un:  0.0 % de nulos


Internet_Accesos_velocidad tiene un:  1.0 % de nulos


Internet_BAF tiene un:  0.0 % de nulos


Internet_ingresos tiene un:  0.0 % de nulos


Internet_Penetracion tiene un:  0.0 % de nulos


Listado_localidades_conectividad_internet tiene un:  0.0 % de nulos


velocidadInternet_anio tiene un:  2.0 % de nulos
